In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import yfinance as yf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Gupta_Text/Portfolio/df.csv') # tone & drift(mom)
fin_q = pd.read_csv('/content/drive/MyDrive/Gupta_Text/Portfolio/quarterly data.csv')# quarterly financial statements
date = pd.read_csv('/content/drive/MyDrive/Gupta_Text/data.csv') # meeting date
df.head()

Mounted at /content/drive


,tiker,drift,negative tone
0,MMM,0.351239,-0.124528
1,AOS,0.042917,-0.206057
2,ABT,0.374046,-0.071820
3,ABBV,0.280534,-0.237878
4,ACN,0.136894,-0.260297


In [ ]:
fin_q['Totaldebt'] = fin_q['dlcq'] + fin_q['dlttq'] # ST + LT debt
fin_q['leverage'] = fin_q['Totaldebt'] / fin_q['atq']
fin_q['profitbility'] = fin_q['niq']/fin_q['revtq']
fin_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40338 entries, 0 to 40337
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gvkey         40338 non-null  int64  
 1   datadate      40338 non-null  object 
 2   fyearq        40338 non-null  int64  
 3   fqtr          40329 non-null  float64
 4   fyr           40338 non-null  int64  
 5   indfmt        40338 non-null  object 
 6   consol        40338 non-null  object 
 7   popsrc        40338 non-null  object 
 8   datafmt       40338 non-null  object 
 9   tic           40338 non-null  object 
 10  cusip         40338 non-null  object 
 11  conm          40338 non-null  object 
 12  curcdq        40338 non-null  object 
 13  datacqtr      40318 non-null  object 
 14  datafqtr      40317 non-null  object 
 15  atq           39518 non-null  float64
 16  dlcq          36869 non-null  float64
 17  dlttq         39305 non-null  float64
 18  niq           39989 non-nu

In [ ]:
na_counts_l = fin_q['leverage'].isna().groupby(fin_q['tic']).sum()
print(na_counts_l)
na_counts_p = fin_q['profitbility'].isna().groupby(fin_q['tic']).sum()
print(na_counts_p)

tic
A        2
AAL      0
AAPL     0
ABBV    10
ABNB     8
        ..
XYL      7
YUM      0
ZBH      5
ZBRA     0
ZTS      8
Name: leverage, Length: 497, dtype: int64
tic
A       0
AAL     0
AAPL    0
ABBV    3
ABNB    0
       ..
XYL     4
YUM     0
ZBH     0
ZBRA    0
ZTS     6
Name: profitbility, Length: 497, dtype: int64


In [ ]:
lev_q = fin_q[['gvkey','datadate','tic','atq','Totaldebt','leverage']]
lev_q = lev_q.merge(df, left_on='tic', right_on='tiker', how='inner')
lev_q['tone_lev'] = lev_q['negative tone']*lev_q['leverage']
lev_q.head()

,gvkey,datadate,tic,atq,Totaldebt,leverage,tiker,drift,negative tone,tone_lev
0,1045,2000-03-31,AAL,24314.0,6016.0,0.247429,AAL,0.591604,-0.108493,-0.026844
1,1045,2000-06-30,AAL,25473.0,6137.0,0.240922,AAL,0.591604,-0.108493,-0.026138
2,1045,2000-09-30,AAL,25899.0,5966.0,0.230356,AAL,0.591604,-0.108493,-0.024992
3,1045,2000-12-31,AAL,26213.0,6270.0,0.239194,AAL,0.591604,-0.108493,-0.025951
4,1045,2001-03-31,AAL,26534.0,6573.0,0.247720,AAL,0.591604,-0.108493,-0.026876


In [ ]:
pro_q = fin_q[['gvkey','datadate','tic','atq','profitbility','niq','revtq']]
pro_q = pro_q.merge(df, left_on='tic', right_on='tiker', how='inner')
pro_q['tone_lev_p'] = pro_q['negative tone']*pro_q['profitbility']
pro_q.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,gvkey,datadate,tic,atq,profitbility,niq,revtq,tiker,drift,negative tone,tone_lev_p
0,1045,2000-03-31,AAL,24314.0,0.028840,132.0,4577.0,AAL,0.591604,-0.108493,-0.003129
1,1045,2000-06-30,AAL,25473.0,0.064059,321.0,5011.0,AAL,0.591604,-0.108493,-0.006950
2,1045,2000-09-30,AAL,25899.0,0.059551,313.0,5256.0,AAL,0.591604,-0.108493,-0.006461
3,1045,2000-12-31,AAL,26213.0,0.009673,47.0,4859.0,AAL,0.591604,-0.108493,-0.001049
4,1045,2001-03-31,AAL,26534.0,-0.009034,-43.0,4760.0,AAL,0.591604,-0.108493,0.000980


In [ ]:
# merge new tone_leverage date with meeting in covering quarters
date['Date'] = pd.to_datetime(date['Date'], format='%m/%d/%Y')
date['quarter'] = date['Date'].dt.to_period('Q')
date = date[['Date','quarter']]
lev_q['datadate'] = pd.to_datetime(lev_q['datadate'], format='%Y-%m-%d')  # Adjust format if needed
lev_q['datafqtr'] = lev_q['datadate'].dt.to_period('Q')
lev_q = lev_q.merge(date, left_on='datafqtr', right_on='quarter', how='inner')
lev_q.rename(columns={'datadate': 'report_date','Date':'meeting_date'}, inplace=True)
lev_q.drop(columns=['tic','quarter'], inplace=True)
lev_q.head()

,gvkey,report_date,atq,Totaldebt,leverage,tiker,drift,negative tone,tone_lev,datafqtr,meeting_date
0,1045,2011-06-30,25787.000,11882.000,0.460775,AAL,0.591604,-0.108493,-0.049991,2011Q2,2011-04-27
1,1045,2011-06-30,25787.000,11882.000,0.460775,AAL,0.591604,-0.108493,-0.049991,2011Q2,2011-06-22
2,1075,2011-06-30,12474.080,3672.511,0.294411,PNW,0.818746,-0.381895,-0.112434,2011Q2,2011-04-27
3,1075,2011-06-30,12474.080,3672.511,0.294411,PNW,0.818746,-0.381895,-0.112434,2011Q2,2011-06-22
4,1078,2011-06-30,62004.241,18219.111,0.293837,ABT,0.374046,-0.071820,-0.021103,2011Q2,2011-04-27


In [ ]:
tic = lev_q['tiker'].tolist()
start_date = '2011-02-01'
end_date = '2020-09-30'
price = yf.download(tic, start=start_date, end=end_date)['Adj Close']

[*********************100%%**********************]  485 of 485 completed


In [ ]:
# # use 1/price as position for each stock on each date, deploy when stocks are in portfolio
# weight = 1/price
# weight.head()

In [ ]:
ret = (price/price.shift(1)-1)
# Reset index to have 'date', 'tiker', and 'return' columns
ret = ret.reset_index()
ret_melt = ret.melt(id_vars=['Date'], var_name='tiker', value_name='return')
# weight_melt = weight.reset_index()
# weight_melt = weight_melt.melt(id_vars=['Date'], var_name='tiker', value_name='weight')
# ret_weight  = ret_melt.merge(weight_melt, on=['Date','tiker'], how='inner')
ret_melt.head()

,Date,tiker,return
0,2011-02-01,A,NaN
1,2011-02-02,A,-0.019501
2,2011-02-03,A,-0.005821
3,2011-02-04,A,0.048793
4,2011-02-07,A,0.033729


In [ ]:
lev_q_ret = lev_q.merge(ret_melt, left_on=['meeting_date','tiker'], right_on=['Date','tiker'], how='inner')
lev_q_ret.drop(columns=['Date'], inplace=True)
lev_q_ret.head()

,gvkey,report_date,atq,Totaldebt,leverage,tiker,drift,negative tone,tone_lev,datafqtr,meeting_date,return
0,1045,2011-06-30,25787.000,11882.000,0.460775,AAL,0.591604,-0.108493,-0.049991,2011Q2,2011-04-27,-0.005682
1,1045,2011-06-30,25787.000,11882.000,0.460775,AAL,0.591604,-0.108493,-0.049991,2011Q2,2011-06-22,-0.019653
2,1075,2011-06-30,12474.080,3672.511,0.294411,PNW,0.818746,-0.381895,-0.112434,2011Q2,2011-04-27,0.003953
3,1075,2011-06-30,12474.080,3672.511,0.294411,PNW,0.818746,-0.381895,-0.112434,2011Q2,2011-06-22,-0.005193
4,1078,2011-06-30,62004.241,18219.111,0.293837,ABT,0.374046,-0.071820,-0.021103,2011Q2,2011-04-27,0.007919


In [ ]:
ticker_counts = lev_q_ret.groupby('tiker').size().reset_index(name='count')
ticker_counts

,tiker,count
0,A,46
1,AAL,46
2,AAPL,46
3,ABBV,46
4,ABT,46
...,...,...
480,XYL,46
481,YUM,46
482,ZBH,46
483,ZBRA,46


In [31]:
def get_long_short(data,feature):
    df_sorted = data.sort_values(by=feature)
    num_samples = len(df_sorted)
    top_tail_count = int(num_samples * 0.4)
    long_10_percent_t = df_sorted.tail(top_tail_count)
    short_10_percent_t = df_sorted.head(top_tail_count)
    # Add a column indicating whether it's long or short
    long_10_percent_t['position'] = 1
    short_10_percent_t['position'] = -1
    # Combine long and short positions into one DataFrame
    combined_df = pd.concat([long_10_percent_t, short_10_percent_t])
    return combined_df

# Dictionary to store results
results_dict = {}

# Populate the dictionary with meeting_date as key and combined DataFrame as value
for date, group in lev_q_ret.groupby('meeting_date'):
  formatted_date = date.strftime('%Y-%m-%d')  # Format the date to remove time
  results_dict[formatted_date] = get_long_short(group,'tone_lev')

<ipython-input-31-201ae4eefc8e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_10_percent_t['position'] = 1
<ipython-input-31-201ae4eefc8e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_10_percent_t['position'] = -1
<ipython-input-31-201ae4eefc8e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [32]:
results_list = []

for date, result in results_dict.items():
    print(f"Date: {date}")
    long_positions = result[result['position'] == 1]
    short_positions = result[result['position'] == -1]
    ret_sum_long = (result['position'] * result['return']).sum()/ len(long_positions)
    ret_sum_short = (result['position'] * result['return']).sum()/ len(short_positions)
    ret_sum = ret_sum_long + ret_sum_short
    print(f"Total return equals position * return, daily: {ret_sum}")
    print("\n")
    print(result[['meeting_date','tiker','position','negative tone','tone_lev','return']])
    print("\n")
    results_list.append({'Date': date, 'Total Return': ret_sum})

Date: 2011-04-27
Total return equals position * return, daily: 0.01005428594829824


    meeting_date tiker  position  negative tone  tone_lev    return
578   2011-04-27    CB         1      -0.183121 -0.010400  0.012544
572   2011-04-27   HUM         1      -0.108034 -0.010217 -0.011344
78    2011-04-27   CHD         1      -0.124303 -0.010159  0.008497
458   2011-04-27   NVR         1      -0.225450 -0.010143  0.024120
490   2011-04-27   WRB         1      -0.100639 -0.010027  0.006291
..           ...   ...       ...            ...       ...       ...
906   2011-04-27  TSLA        -1      -0.162073 -0.033817  0.005570
232   2011-04-27  KLAC        -1      -0.209417 -0.033426  0.002730
444   2011-04-27  JKHY        -1      -0.323165 -0.033057  0.003867
138   2011-04-27   EMR        -1      -0.151706 -0.032822  0.007553
452   2011-04-27  MSFT        -1      -0.295772 -0.032436  0.007254

[370 rows x 6 columns]


Date: 2011-06-22
Total return equals position * return, daily: -0.0004763

In [33]:
# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results_list)
results_df.to_csv('port_results.csv', index=False)

In [34]:
results_df.describe()

,Total Return
count,46.000000
mean,-0.000615
std,0.010223
min,-0.023637
25%,-0.006618
50%,-0.000582
75%,0.005311
max,0.032852
